## InfluxDB Python Client
* 메뉴얼 : http://influxdb-python.readthedocs.io/en/latest/
* 클라이언트 다운로드 : https://github.com/influxdata/influxdb-python

In [1]:
from influxdb import InfluxDBClient

In [2]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'telegraf')
# client.create_database('example2')

In [3]:
json_body = [
    {
        "measurement": "cpu_load_short",
        "tags": {
            "host": "server01",
            "region": "us-west"
        },
        "time": "2009-11-10T23:00:00Z",
        "fields": {
            "value": 0.64
        }
    }
]

In [4]:
client.write_points(json_body)

True

In [19]:
import argparse
import pandas as pd

from influxdb import DataFrameClient


def pandas_test(host='localhost', port=8086):
    user = 'root'
    password = 'root'
    dbname = 'example'

    client = DataFrameClient(host, port, user, password, dbname)

    print("Create pandas DataFrame")
    df = pd.DataFrame(data=list(range(100)),
                      index=pd.date_range(start='2016-08-29',
                                          periods=100, freq='H'))

    print("Write DataFrame")
    client.write_points(df, 'demo')

    print("Write DataFrame with Tags")
    client.write_points(df, 'demo', {'k1': 'v1', 'k2': 'v2'})

    print("Read DataFrame")
    client.query("select * from demo")
    
    print(df)

    
pandas_test()

Create pandas DataFrame
Write DataFrame
Write DataFrame with Tags
Read DataFrame
                            0
2016-08-29 00:00:00+00:00   0
2016-08-29 01:00:00+00:00   1
2016-08-29 02:00:00+00:00   2
2016-08-29 03:00:00+00:00   3
2016-08-29 04:00:00+00:00   4
2016-08-29 05:00:00+00:00   5
2016-08-29 06:00:00+00:00   6
2016-08-29 07:00:00+00:00   7
2016-08-29 08:00:00+00:00   8
2016-08-29 09:00:00+00:00   9
2016-08-29 10:00:00+00:00  10
2016-08-29 11:00:00+00:00  11
2016-08-29 12:00:00+00:00  12
2016-08-29 13:00:00+00:00  13
2016-08-29 14:00:00+00:00  14
2016-08-29 15:00:00+00:00  15
2016-08-29 16:00:00+00:00  16
2016-08-29 17:00:00+00:00  17
2016-08-29 18:00:00+00:00  18
2016-08-29 19:00:00+00:00  19
2016-08-29 20:00:00+00:00  20
2016-08-29 21:00:00+00:00  21
2016-08-29 22:00:00+00:00  22
2016-08-29 23:00:00+00:00  23
2016-08-30 00:00:00+00:00  24
2016-08-30 01:00:00+00:00  25
2016-08-30 02:00:00+00:00  26
2016-08-30 03:00:00+00:00  27
2016-08-30 04:00:00+00:00  28
2016-08-30 05:00:00

client.create_user('koon', 'koon', admin=True)

In [3]:
client.create_user('telegraf', 'telegraf', admin=True)

In [6]:
client.get_list_database()

[{'name': '_internal'},
 {'name': 'mydb'},
 {'name': 'example'},
 {'name': 'example2'}]

In [7]:
client.get_list_users()

[{'admin': True, 'user': 'koon'}]

In [8]:
client.query('select * from cpu_load_short', database='example')

ResultSet({'('cpu_load_short', None)': [{'host': 'server01', 'value': 0.64, 'region': 'us-west', 'time': '2009-11-10T23:00:00Z'}]})

AttributeError: 'InfluxDBClient' object has no attribute 'change_database'

In [23]:
import time
from influxdb import InfluxDBClient

class InfluxDB :
    m_conn = None
    def __init__(self) :
        pass

    def open(self) :
        bConti = True
        while bConti :
            try :
                self.m_dConn = InfluxDBClient('localhost', 8086, 'koon', 'koon', 'mydb')
                bConti = False
            except Exception:
                time.sleep(CmpGlobal.g_nConnectionRetryInterval)
                print("--------------------------------- influxdb connect fail -------------------------------------")
                
    def insertData(self, jsondata) :
        print("Write points: {0}".format(jsondata))
        self.m_dConn.write_points(jsondata)

class InfluxDBManager :
    m_oDBConn = None
    
    def __init__(self) :
        self.m_oDBConn = InfluxDB()
        self.m_oDBConn.open()

    def insert(self, nNodeId, watt, apc, pfc, pfcval, ipeak, vpeak) :
        json_meter_body = [
            {
                "measurement": "officepower",
                "tags": {
                    "node":  nNodeId
                },
                "fields": {
                    "watt":  watt,
                    "apc":  apc,
                    "pfc":  pfc,
                    "pfcval": pfcval,
                    "ipeak" : ipeak,
                    "vpeak" : vpeak
                }
            }
        ]
        self.m_oDBConn.insertData(json_meter_body)
        return 0

In [28]:
g_influxdbconn = InfluxDBManager()

# 아래와 같이 입력함.
#  g_influxdbconn.insert(nNodeId,wattVal,dWattFactor,dPowerFactor,pftVal,iPeak,vPeak)

In [29]:
g_influxdbconn.insert('Host01', 300, 20, 10, 1, 30, 20)

Write points: [{'tags': {'node': 'Host01'}, 'measurement': 'officepower', 'fields': {'pfcval': 1, 'watt': 300, 'ipeak': 30, 'pfc': 10, 'vpeak': 20, 'apc': 20}}]


0

### DataframeClient with Pandas 예제
* https://github.com/influxdata/influxdb-python/tree/master/examples

In [36]:
import argparse
import pandas as pd

from influxdb import DataFrameClient


def main(host='localhost', port=8086):
    user = 'root'
    password = 'root'
    dbname = 'example3'

    client = DataFrameClient(host, port, user, password, dbname)

    print("Create pandas DataFrame")
    df = pd.DataFrame(data=list(range(30)),
                      index=pd.date_range(start='2014-11-16',
                                          periods=30, freq='H'))

    print("Create database: " + dbname)
    client.create_database(dbname)

    print("Write DataFrame")
    client.write_points(df, 'demo')

    print("Write DataFrame with Tags")
    client.write_points(df, 'demo', {'k1': 'v1', 'k2': 'v2'})

    print("Read DataFrame")
    client.query("select * from demo")

    #print("Delete database: " + dbname)
    #client.drop_database(dbname)


def parse_args():
    parser = argparse.ArgumentParser(
        description='example code to play with InfluxDB')
    parser.add_argument('--host', type=str, required=False,
                        default='localhost',
                        help='hostname of InfluxDB http API')
    parser.add_argument('--port', type=int, required=False, default=8086,
                        help='port of InfluxDB http API')
    return parser.parse_args()

if __name__ == '__main__':
    main()
    #args = parse_args()
    #main(host=args.host, port=args.port)

Create pandas DataFrame
Create database: example3
Write DataFrame
Write DataFrame with Tags
Read DataFrame
